Examining and Visualizing Data
=============================

<div class="alert alert-block alert-info"> 
<h2>Overview</h2>

<strong>Questions:</strong>

* How can I use pandas to process data?

* How can I visualize relationships between different parts of my data?

<strong>Objectives:</strong>

* Use pandas and seaborn to load and explore data

</div>

[Pandas](https://pandas.pydata.org/docs/) is a Python library used for data analysis and manipulation. Within the world of data science, it is a ubiquitous and widely used library. If you are learning how to analyze data in Python, it will be almost impossible to avoid pandas. 

The central data structure of pandas is called a DataFrame. Pandas DataFrames work very closely with NumPy arrays and Pandas dataframes are specifically for data which is two dimensional (rows and columns). NumPy arrays, while similar in some ways, can work with higher dimensional data. 

Pandas is very powerful. In this session, we'll be learning how to access information in pandas dataframes and how to do some basic manipulation and analysis. We are going to be looking at a dataset which gives information about the elements in the periodic table.

In [ ]:
import pandas as pd
df = pd.read_csv("data/PubChemElements_all.csv")

## Examining Data

Initially when loading data in, and also at certain points as we're working with it, we'll want to see what our dataframe looks like. Youo can see a preview of your dataframe using the `.head` function

In [ ]:
df.head()

The `.info` function will give information about the columns and the data type of those columns. The data type will become very important later as we work with data more.

In [ ]:
df.info()

For this dataframe, we see that the first column, `AtomicNumber` has the data type of `int64`. Here, `int` means `integer` and `64` means `64 bit`.  The `64 bit` refers to the amount of computer memory the variable can occupy. It won't really be important for us. Similarly, `float64` means `64 bit floating point`. These are decimal numbers.

The other column names which read `object` are not numeric. They might be strings or they might be something else. We'll discuss more later.

The `describe` function can be used on a dataframe to quickly see statistics about columns with numerical data. If you look at the columns that statistics are computed for and compare to the data type shown from `info`, you will see that we only get statistics for columns which had `int64` or `float64` data types.

In [ ]:
df.describe()

This information is extremely useful for understanding the data. We can also easily visualize the distribution of each column using Pandas's ``hist`` function.

In [ ]:
df.hist(figsize=(8,8), edgecolor='black', grid=False)

## Accessing Data

Pandas dataframes have names for rows (called the "index" in Pandas) and columns.

Pandas dataframes have rows and columns, you can see how many rows and columns using `.shape`. This will return the shape as `(num_rows, num_columns)`.

In [ ]:
df.shape

There are a few methods for accessing information in a Pandas dataframe, but the one that will be most important in this workshop is selecting particular columns of data.

In [ ]:
df["AtomicNumber"].head()

In [ ]:
df[["Symbol", "ElectronConfiguration"]].head() 

## Performing calculations with pandas: No more `for` loops!

Both pandas and NumPy dataframes have the convenient feature that they can do element-wise operations and use something called `broadcasting`. This means that if you are doing something like subtracting a number, multiplying, etc to a column or dataframe of information, it can be done all at once instead of with a `for` loop. Consider if we wanted to calculate the melting point in degrees celsius for all of the elements.

Instead of writing a `for` loop that does this, we can just write the following code. This will return a pandas Series (one dimensional dataframe). 

In [ ]:
df['MeltingPoint'] - 273.15

We could do this one two columns as well.

In [ ]:
df[['MeltingPoint', 'BoilingPoint']] - 273.15

We can save these in new dataframe columns

In [ ]:
df[["MeltingPointC", "BoilingPointC"]] = df[['MeltingPoint', 'BoilingPoint']] - 273.15

### The `.apply` method

The `.apply` method in pandas is used to apply a function along a row or column of a dataframe.
This is useful when you have a custom function that you need to use on every value in a column, but there is not a NumPy or Pandas function for it.

For example, we could apply the `len` function to our `Name` column to get the number of letters in the name for each element.

In [ ]:
# Number of letters in name - 
# point - you can use built in functions
df["Name"].apply(len)

For an example more related to our work with RDKit, let's add some additional atomic data.
RDKit has the ability to get information about atoms.
We can create a periodic table with `Chem.GetPeriodicTable`, then use associated functions to get information about atoms.

In [ ]:
from rdkit import Chem

# Initialize the periodic table
periodic_table = Chem.GetPeriodicTable()

In [ ]:
df["NOuter"] = df["Symbol"].apply(periodic_table.GetNOuterElecs)
df.head()

<div class="alert alert-block alert-warning">
<h3>Exercise</h3>

Use the `tab` key on your periodic table object to check for other values you can calculate for atoms.
Pick one to add to your periodic table dataset.

</div>

In [ ]:
# pick default valence
df["DefaultValence"] = df["NOuter"].apply(periodic_table.GetDefaultValence)
df.head()

## Visualization

Visualizing data helps in understanding relationships and patterns that might not be apparent from raw data. Here, we will use Seaborn, a statistical visualization library, to create plots from our periodic table dataset. Seaborn is built on top of matplotlib, so if we would like to adjust any of the plots seaborn makes, we can do that through the Matplotlib interface we've used before.

We will start with a bar plot to show the ionization energy of elements across different group blocks:


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.catplot(data=df, x="NOuter", y="IonizationEnergy", kind="bar")
# Rotate x-axis labels
plt.xticks(rotation=45, ha='right')

The plot above shows us periodic trends that we learned about in introductory chemistry. The two highest ionization energy categories correspond to elements with 2 valence electrons and 8 valence electrons, representing filled shells.

Seaborn can also allow us to easily create scatter plots to visualize relationships between continuous variables. For example, we can create a scatter plot to show the relationship between ionization energy and atomic radius:

In [ ]:
sns.scatterplot(data=df, x="AtomicRadius", y="Electronegativity", hue="GroupBlock")
plt.title('Electronegativity vs. Atomic Radius')
plt.xlabel('Atomic Radius')
plt.ylabel('Electronegativity')
plt.show()

<div class="alert alert-block alert-warning">
<h3>Exercise</h3>

Create a few other categorical plots to observe periodic trends:

1. Electronegativity vs. Group Block as a bar plot.

2. Melting Point vs. Group Block as a bar plot.

3. Ionization Energy vs. Atomic Number as a scatter plot colored by GroupBlock.
   
</div>

In [ ]:
sns.catplot(data=df, x="GroupBlock", y="Electronegativity", kind="bar")
plt.title('Electronegativity vs. Group Block')
plt.xlabel('Group Block')
plt.ylabel('Electronegativity')
plt.xticks(rotation=45)

In [ ]:
sns.catplot(data=df, x="GroupBlock", y="MeltingPoint", kind="bar")
plt.title('Melting Point vs. Group Block')
plt.xlabel('Group Block')
plt.ylabel('Melting Point')
plt.xticks(rotation=45)

In [ ]:
sns.scatterplot(data=df, x="AtomicNumber", y="IonizationEnergy", hue="GroupBlock")
plt.title('Ionization Energy vs. Atomic Number')
plt.xlabel('Atomic Number')
plt.ylabel('Ionization Energy')

### Visualizing Correlation

A common way to visualize relationships between different categories of data categories is with a correlation plot.
The correlation matrix provides insights into the relationships between the variables. A correlation value close to 1 indicates a strong positive relationship, while a correlation value close to -1 indicates a strong negative relationship. A correlation value close to 0 indicates no relationship between the features.

In [ ]:
# Calculate the correlation matrix
corr = df.corr(numeric_only=True)
corr

Seaborn can be used to create a heatmap to allow easier examination of the correlation of different variables.

In [ ]:
# Create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")

The heatmap uses a "coolwarm" color scheme where red indicates positive correlation and blue indicates negative correlation between variables. Strongly correlated pairs are represented by darker shades of red, while strongly inversely correlated pairs are represented by darker shades of blue.

<div class="alert alert-block alert-warning">
<h3>Final Challenge</h3>

Repeat your homework from last night using Pandas and the `.apply` function.
The creation of a starting dataframe is done for you.
You should use the `apply` function to add new columns of data to your DataFrame.

Compare and contrast the code - do you prefer one over the other?


</div>

In [ ]:
from rdkit import Chem
from rdkit.Chem import PandasTools

# Ensure molecules are rendered in the notebook
PandasTools.RenderImagesInAllDataFrames(images=True)

# Read SMILES strings from a file
with open("data/amino_acids.txt", "r") as f:
    amino_acid_smiles = f.readlines()

df = pd.DataFrame(amino_acid_smiles, columns=["SMILES"])

PandasTools.AddMoleculeColumnToFrame(df,'SMILES','Molecule',includeFingerprints=True)

df.head(3)

In [ ]:
df["MolWt"] = df["Molecule"].apply(Descriptors.MolWt)
df["NumHeavy"] = df["Molecule"].apply(lambda x: x.GetNumHeavyAtoms())
df["TPSA"] = df["Molecule"].apply(Descriptors.rdMolDescriptors.CalcTPSA)

df.head(3)